In [1]:
import pandas as pd 
import numpy as np
from matplotlib.ticker import FuncFormatter
import csv
import requests
from bs4 import BeautifulSoup as bs


In [2]:
url="https://www.spotrac.com/nfl/contracts/quarterback/" #Getting salary data by scraping with Beautiful Soup
page=requests.get(url)
soup=bs(page.content,"lxml")

In [3]:
salary=soup.find_all(class_='right xs-hide')
names=soup.find_all(class_='player')

In [4]:
del names[0] #Removing headers
del salary[0:3]
players=[]
salaries=[]

In [5]:
for n in names: #Building list of clean player names
    x=n.children
    next(x)
    name=next(x)
    players.append(name.text)

In [6]:
for i in range(len(salary)):  #Building list of clean salaries. We want average yearly salary so take every third figure
    if (i+3) % 3 == 0:
        salaries.append(salary[i].contents[0])

In [7]:
len(players)==len(salaries) #Test that lists are in correspondence

True

In [8]:
clean_salaries=[] #Get rid of dollars signs and commas and convert to integer
for s in salaries:
    sl=s.replace("$","")
    sal=sl.replace(",","")
    num=int(sal)
    clean_salaries.append(num)
    

In [9]:
salary_df=pd.DataFrame({"Player":players,"Salary":clean_salaries}) #Putting both lists into a dataframe
salary_df.head()

,Player,Salary
0,Matt Ryan,30000000
1,Jimmy Garoppolo,27500000
2,Matthew Stafford,27000000
3,Aaron Rodgers,33500000
4,Derek Carr,25000000


In [10]:
#Save dataframe for presentation notebook
salary_df.to_csv('Resources/Cleaned_Dataframes/salary_df.csv',columns=salary_df.columns,index=False)

In [11]:
#Reading NFL Stats from downloaded CSV
nfl_stats = "Resources/NFL_Stats.csv"
nfl_stats_df = pd.read_csv(nfl_stats)
nfl_stats_df.head()

,Rk,Player,From,To,Draft,Tm,Lg,Ht,Wt,BMI,...,Rate,Y/A,Y/G,W,L,T,Yrs,PB,AP1,AV
0,1,Peyton Manning,1998,2015,1.0,TOT,NFL,77,230,27.3,...,96.5,7.67,270.5,186.0,79.0,0.0,17,14,7,271
1,2,Tom Brady,2000,2018,6.0,NWE,NFL,80,225,27.4,...,97.6,7.51,261.5,196.0,55.0,0.0,19,13,3,255
2,3,Drew Brees,2001,2018,2.0,TOT,NFL,72,209,28.3,...,96.7,7.58,282.9,142.0,106.0,0.0,18,11,1,239
3,4,Brett Favre*,1991,2010,2.0,TOT,NFL,74,225,28.9,...,86.0,7.06,237.9,186.0,112.0,0.0,20,11,3,255
4,5,John Elway*,1983,1998,1.0,DEN,NFL,75,215,26.9,...,79.9,7.10,220.0,148.0,82.0,1.0,16,9,0,203


In [12]:
# Clean Dataframe by renaming and deleting unnecssary columns

nfl_stats_df = nfl_stats_df.rename(columns={"Tm":"Team",
                                           "PB":"Pro Bowls",
                                           "Ht": "Height (in)"})
del nfl_stats_df["Wt"]
del nfl_stats_df["BMI"]
del nfl_stats_df["AV"]
del nfl_stats_df["AP1"]
del nfl_stats_df["Rk"]
del nfl_stats_df["Lg"]
del nfl_stats_df["Team"]

In [13]:
#Get Winning Percentage
nfl_stats_df ["W"].fillna(0, inplace=True)
nfl_stats_df ["L"].fillna(0, inplace=True)
nfl_stats_df ["T"].fillna(0, inplace=True)

nfl_stats_df["Win Record"] = (nfl_stats_df["W"]/(nfl_stats_df["W"] + nfl_stats_df["L"] + nfl_stats_df["T"]))

nfl_stats_df.head()

,Player,From,To,Draft,Height (in),G,GS,Cmp,Att,Cmp%,...,Int,Rate,Y/A,Y/G,W,L,T,Yrs,Pro Bowls,Win Record
0,Peyton Manning,1998,2015,1.0,77,266,265,6125,9380,65.30,...,251,96.5,7.67,270.5,186.0,79.0,0.0,17,14,0.701887
1,Tom Brady,2000,2018,6.0,80,253,251,5629,8805,63.93,...,160,97.6,7.51,261.5,196.0,55.0,0.0,19,13,0.780876
2,Drew Brees,2001,2018,2.0,72,249,248,6222,9294,66.95,...,228,96.7,7.58,282.9,142.0,106.0,0.0,18,11,0.572581
3,Brett Favre*,1991,2010,2.0,74,302,298,6300,10169,61.95,...,336,86.0,7.06,237.9,186.0,112.0,0.0,20,11,0.624161
4,John Elway*,1983,1998,1.0,75,234,231,4123,7250,56.87,...,226,79.9,7.10,220.0,148.0,82.0,1.0,16,9,0.640693


In [14]:
# Fill undrafted with a 0
nfl_stats_df["Draft"].fillna(0, inplace=True)
nfl_stats_df["Win Record"].fillna(0, inplace=True)
nfl_stats_df["Draft"] = nfl_stats_df["Draft"].astype(str)

In [15]:
#Dataframe of each Draft Round

undrafted = nfl_stats_df.loc[nfl_stats_df["Draft"] == "0.0"]
one = nfl_stats_df.loc[nfl_stats_df["Draft"] == "1.0"]
two = nfl_stats_df.loc[nfl_stats_df["Draft"] == "2.0"]
three = nfl_stats_df.loc[nfl_stats_df["Draft"] == "3.0"]
four = nfl_stats_df.loc[nfl_stats_df["Draft"] == "4.0"]
five = nfl_stats_df.loc[nfl_stats_df["Draft"] == "5.0"]
six = nfl_stats_df.loc[nfl_stats_df["Draft"] == "6.0"]
seven = nfl_stats_df.loc[nfl_stats_df["Draft"] == "7.0"]
eight = nfl_stats_df.loc[nfl_stats_df["Draft"] == "8.0"]
nine = nfl_stats_df.loc[nfl_stats_df["Draft"] == "9.0"]
ten = nfl_stats_df.loc[nfl_stats_df["Draft"] == "10.0"]
eleven = nfl_stats_df.loc[nfl_stats_df["Draft"] == "11.0"]
twelve = nfl_stats_df.loc[nfl_stats_df["Draft"] == "12.0"]
thirtheen = nfl_stats_df.loc[nfl_stats_df["Draft"] == "13.0"]
fourtheen = nfl_stats_df.loc[nfl_stats_df["Draft"] == "14.0"]

In [16]:
#Build Dataframe organized by Round and Average Passer Rating

ratings_by_draft = pd.DataFrame({"Undrafted":[undrafted["Rate"].mean()],
                                 "1st Round" : [one["Rate"].mean()],
                                 "2nd Round" : [two["Rate"].mean()],
                                 "3rd Round" : [three["Rate"].mean()],
                                 "4th Round" : [four["Rate"].mean()],
                                 "5th Round" : [five["Rate"].mean()],
                                 "6th Round" : [six["Rate"].mean()],
                                 "7th Round" : [seven["Rate"].mean()],
                                 "8th Round" : [eight["Rate"].mean()],
                                 "9th Round" : [nine["Rate"].mean()],
                                 "10th Round" : [ten["Rate"].mean()],
                                 "11th Round" : [eleven["Rate"].mean()],
                                 "12th Round" : [twelve["Rate"].mean()],
                                 "13th Round" : [thirtheen["Rate"].mean()],
                                 "14th Round" : [fourtheen["Rate"].mean()]})

ratings_by_draft = ratings_by_draft.T
ratings_by_draft.reset_index(level=0, inplace = True)
ratings_by_draft = ratings_by_draft.rename(columns={"index": "Draft Round",
                                                   "0": "Passer Rating"})

In [17]:
#Save dataframes for presentation notebook
nfl_stats_df.to_csv("Resources/Cleaned_Dataframes/nfl_stats_df.csv",columns=nfl_stats_df.columns,index=False)
ratings_by_draft.to_csv("Resources/Cleaned_Dataframes/ratings_by_draft.csv",columns=ratings_by_draft.columns,index=False)

In [18]:
reduced_nfl_stats_df = nfl_stats_df.loc[:,["Player","Draft", "Rate", "Pro Bowls", "Win Record"]]
grpby_draft = reduced_nfl_stats_df.groupby("Draft", as_index=False)
average_rate = pd.DataFrame(grpby_draft ["Rate"].mean())
average_rate

,Draft,Rate
0,0.0,68.410833
1,1.0,75.725962
2,10.0,66.225000
3,11.0,57.044444
4,12.0,55.160000
5,13.0,79.100000
6,14.0,64.200000
7,2.0,68.637209
8,3.0,71.063265
9,4.0,68.622642


In [19]:
percent_nfl_stats_df=reduced_nfl_stats_df.dropna()
percent_nfl_stats_df["Win Record"] = percent_nfl_stats_df["Win Record"]*100
percent_nfl_stats_df.head()

,Player,Draft,Rate,Pro Bowls,Win Record
0,Peyton Manning,1.0,96.5,14,70.188679
1,Tom Brady,6.0,97.6,13,78.087649
2,Drew Brees,2.0,96.7,11,57.258065
3,Brett Favre*,2.0,86.0,11,62.416107
4,John Elway*,1.0,79.9,9,64.069264


In [20]:
#Save dataframes for presentation notebook
percent_nfl_stats_df.to_csv("Resources/Cleaned_Dataframes/percent_nfl_stats_df.csv",columns=percent_nfl_stats_df.columns,index=False)
average_rate.to_csv("Resources/Cleaned_Dataframes/average_rate.csv",columns=average_rate.columns,index=False)

In [21]:
#Get and clean college football players and ratings
player=[]
rate=[]
for year in range(1982,2018):
    file="CFB/"+str(year)+".csv"
    with open(file,newline='') as f:
        reader=csv.reader(f)
        next(reader)
        next(reader)
        for row in reader:
            bad_name=row[1].replace("*","") #Cleaning player names
            split_names=bad_name.split("\\")
            clean_name=split_names[0]                          
            player.append(clean_name)
            rate.append(float(row[13]))

In [22]:
len(player)==len(rate) #Test

True

In [23]:
#Go backwards to get only last year of college for each player
players=[]
ratings=[]  
for i in range(len(player)-1,-1,-1): 
    if player[i] not in players:
        players.append(player[i])
        ratings.append(rate[i])

In [24]:
len(players)==len(ratings)#test

True

In [25]:
college=pd.DataFrame({"Player":players,"College Rating":ratings})
college.head()

,Player,College Rating
0,Anthony Brown,103.5
1,Hasaan Klugh,96.1
2,Max Bortenschlager,109.1
3,Steven Williams,103.9
4,Peyton Bender,108.5


In [26]:
names=[]
qbr=[]
wins=[]
loss=[]
pbs=[]
winper=[]
games=[]

In [27]:
with open("Resources/NFL_Stats.csv",newline="") as nf: #NFL data to be merged with college data
    read=csv.reader(nf)
    next(read)
    for r in read:
        names.append(r[1].replace("*","")) #Clean asterisk from names
        qbr.append(float(r[18]))
        try:
            wins.append(int(r[21]))
        except:
            wins.append("")
        try:
            loss.append(int(r[22]))
        except:
            loss.append("")
        try:
            per=round((int(r[21])/(int(r[21])+int(r[22])))*100,1)
            winper.append(per)         
        except:
            winper.append("")
        try:
            games.append(int(r[21])+int(r[22]))
        except:
            games.append("")
        pbs.append(int(r[25]))

In [28]:
len(names)==len(qbr)&len(wins)==(len(loss)&len(pbs)) #Test

True

In [29]:
nfltmp=pd.DataFrame({"Player":names,"NFL QBR":qbr,"Games":games,"Wins":wins,"Losses":loss,"Win Percentage":winper})
nfltmp.head()

,Player,NFL QBR,Games,Wins,Losses,Win Percentage
0,Peyton Manning,96.5,265,186,79,70.2
1,Tom Brady,97.6,251,196,55,78.1
2,Drew Brees,96.7,248,142,106,57.3
3,Brett Favre,86.0,298,186,112,62.4
4,John Elway,79.9,230,148,82,64.3


In [30]:
#Save dataframes for presentation notebook
college.to_csv("Resources/Cleaned_Dataframes/college.csv",columns=college.columns,index=False)
nfltmp.to_csv("Resources/Cleaned_Dataframes/nfltmp.csv",columns=nfltmp.columns,index=False)